In [1]:
import torch
import torch.nn as nn
import os
import sys
import numpy as np
import math

import matplotlib.pyplot as plt
import cv2

import matplotlib as mpl

mpl.rcParams['figure.dpi'] = 300
#plt.rcParams['text.usetex'] = True
#plt.rcParams['font.family'] = 'serif'
#plt.rcParams['font.serif'] = ['Computer Modern']
#plt.rcParams['font.size'] = 12

In [2]:
## Load Trained model

nn_depth = 60
nn_width = 4

x_dim = 2
output_dim = 2
is_unsteady = True
num_param = 2

input_dim = x_dim + is_unsteady + num_param

# X[:,0], X[:,1] space dimensions
# X[:,2] time dimension
# X[:,3] viscosity
# X[:,4] input speed

L_height = 0.5
Tmax = 1

loadmodel = 'flowpastdisk_hyper'

imagefilename = 'hyper_mu02_v8'
imagesfolder = "images/hyperparameter/"

video_name = imagesfolder + imagefilename + '.avi'
fourcc = cv2.VideoWriter_fourcc(*'MP4V') 

In [3]:
class IncompressibleNN(torch.nn.Module):
    
    ### Incompressible neural network
    ### curl operation built in
    
    def __init__(self, depth, width, x_dim, is_unsteady, num_param, **nn_param):
        super(IncompressibleNN, self).__init__()
        
        self.x_dim = x_dim
        self.input_dim = self.x_dim + is_unsteady + num_param
        
        self.dim_out = [1, 3][self.x_dim==3]
        
        modules = []
        modules.append(torch.nn.Linear(self.input_dim, depth))
        for i in range(width - 1):
            modules.append(torch.nn.Linear(depth, depth))
            modules.append(torch.nn.Tanh())
        modules.append(torch.nn.Linear(depth, self.dim_out))
                       
        self.sequential_model = torch.nn.Sequential(*modules)
    
    def curl(self, a, x):
        if self.x_dim == 2:
            
            dadx = torch.autograd.grad(a, x, grad_outputs = torch.ones_like(a), 
                                        create_graph = True, retain_graph = True, allow_unused=False)[0]

            u = torch.stack([dadx[:,1], -dadx[:,0]] , dim=1)
            
        elif self.x_dim == 3:
            e = torch.eye(self.x_dim, device=x.device)

            da0dx = torch.autograd.grad(a, x, grad_outputs=e[0,:].repeat(a.size(0), 1), 
                                        create_graph=True, retain_graph = True)[0]
            da1dx = torch.autograd.grad(a, x, grad_outputs=e[1,:].repeat(a.size(0), 1),
                                        create_graph=True, retain_graph = True)[0]
            da2dx = torch.autograd.grad(a, x, grad_outputs=e[2,:].repeat(a.size(0), 1),
                                        create_graph=True, retain_graph = True, allow_unused=True)[0]

            u = torch.stack([da2dx[:,1] - da1dx[:,2], da0dx[:,2] - da2dx[:,0], da1dx[:,0] - da0dx[:,1] ], dim=1)         
        return u
    
    def forward(self, x):
        a = self.sequential_model(x)
        u = self.curl(a, x)
            
        return u

In [4]:
nn_param = {'depth': nn_depth,
            'width': nn_width,
            'x_dim': x_dim,
            'is_unsteady': is_unsteady,
            'output_dim': output_dim,
            'num_param': num_param
            }
                
################ Preparing the model #################

#print("Initializing the model")

### Initialize the Model
model = IncompressibleNN(**nn_param)

model.load_state_dict(torch.load("savedmodels/" + loadmodel + ".pt", map_location=torch.device('cpu')))

<All keys matched successfully>

In [5]:
# Choose parameters

### Choose time step
dt = 0.01
num_t = int( Tmax/dt )
framerate = 5

### Choose your input speed and viscosity
Inputspeed = 8 # 1 to 10
Viscosity = 0.2 # 0.1 to 1

### Make domain

numplotpts_x = 4*16
numplotpts_y = 16
numplotpts = numplotpts_x*numplotpts_y

xg = torch.cartesian_prod(torch.linspace(0, 5*L_height, numplotpts_x), 
                           torch.linspace(-L_height, L_height, numplotpts_y))

domain = torch.cat( (xg, torch.zeros((xg.size(0),1)), torch.tensor([[Viscosity, Inputspeed]])*torch.ones((xg.size(0),2))),dim=1 )

xplot = xg[:,0].reshape(numplotpts_x, numplotpts_y).detach().numpy()
yplot = xg[:,1].reshape(numplotpts_x, numplotpts_y).detach().numpy()

contour_levels= np.linspace(-Inputspeed, 2*Inputspeed, 50)
contour_ticks = [contour_levels[0], 0, contour_levels[-1]]

def plot_contour(ax, x, y, u, ii):

    ax.set_title(r'$u_1$ velocity at time {:.2f}'.format(ii*dt))

    ax.tick_params(
    axis='both',
    which='both',
    bottom=False,
    top=False,
    left=False,
    right=False,
    labelbottom=False,
    labelleft=False
    )
    th = np.linspace(0,2*math.pi, 64)

    xc = L_height/3 * np.cos( th ) + L_height
    yc = L_height/3 * np.sin( th )

    ax.plot(xc,yc, 'k-')

    contour = ax.contourf(x, y, u, levels=contour_levels)
    plt.colorbar(contour, ticks=contour_ticks)

    return ax

In [6]:
plt.close('all')
fig = plt.figure(figsize=(6,3))
plt.tight_layout(pad=0.75)

for ii in range(num_t):

    domain.requires_grad=False
    domain[:,2] = ii*dt

    Trained_Vel = model(domain.requires_grad_(True)).reshape(numplotpts_x, numplotpts_y, 2).detach().numpy()

    ax = fig.add_subplot(1,1,1)

    plot_contour(ax, xplot, yplot, Trained_Vel[:,:,0], ii)

    filename = imagesfolder + imagefilename + str(ii) + ".png"
    if os.path.isfile(filename):
        os.remove(filename)
    fig.savefig(filename)

    if ii==0:
        frame = cv2.imread(imagesfolder + imagefilename + str(ii) + ".png") 
        # setting the frame width, height width 
        # the width, height of first image 
        height, width, layers = frame.shape   
        video = cv2.VideoWriter(video_name, fourcc, framerate, (width, height)) 
        video.write(cv2.imread(imagesfolder + imagefilename + str(ii) + ".png")) 
    else:
        video.write(cv2.imread(imagesfolder + imagefilename + str(ii) + ".png"))  

    #quiver = ax.quiver(xplot,yplot, Trained_Vel[:,:,0], Trained_Vel[:,:,1], pivot='mid')

    fig.clf()

video.release()  # releasing the video generated     
cv2.destroyAllWindows()

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'avi / AVI (Audio Video Interleaved)'
OpenCV: FFMPEG: fallback to use tag 0x34504d46/'FMP4'


<Figure size 1800x900 with 0 Axes>